In [1]:
# Importing necessary packages.
import pandas as pd
import numpy as np

In [2]:
# Reading the data.
df = pd.read_csv (r'/Users/Colin/Desktop/Research project/Sigma/Data - Final/Final Data.csv')
df.head(5)

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,cusip,...,txpq,uaptq,chechy,cogsy,dpy,niy,revty,xopry,xsgay,costat
0,1000,31/03/1970,1970,1.0,INDL,C,D,STD,AE.2,000032102,...,NaN,NaN,NaN,NaN,NaN,0.605,9.478,NaN,NaN,I
1,1000,30/06/1970,1970,2.0,INDL,C,D,STD,AE.2,000032102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I
2,1000,30/09/1970,1970,3.0,INDL,C,D,STD,AE.2,000032102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I
3,1000,31/12/1970,1970,4.0,INDL,C,D,STD,AE.2,000032102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I
4,1000,31/03/1971,1971,1.0,INDL,C,D,STD,AE.2,000032102,...,NaN,NaN,NaN,NaN,NaN,0.346,7.983,NaN,NaN,I


In [3]:
df.columns

Index(['gvkey', 'datadate', 'fyearq', 'fqtr', 'indfmt', 'consol', 'popsrc',
       'datafmt', 'tic', 'cusip', 'conm', 'curcdq', 'datacqtr', 'datafqtr',
       'acoq', 'aoq', 'dlcq', 'invtq', 'lcoq', 'ltq', 'ppegtq', 'rectq',
       'txpq', 'uaptq', 'chechy', 'cogsy', 'dpy', 'niy', 'revty', 'xopry',
       'xsgay', 'costat'],
      dtype='object')

In [4]:
# Function to fill in missing values in a forward manner.
def fillForwardDataframe(df, id):
    # type(df) = pandas DataFrame
    # type(id) = str
    df_ffill = df.copy()
    df_ffill = df_ffill.groupby(id).ffill()
    df_ffill[id] = df[id]
    return df_ffill

# Using function to fill in the missing values in the dataframe. We do this only in a forward manner.
id = 'cusip'
df_ffill = fillForwardDataframe(df, id)

# Verifying output of the function.
df_ffill['txpq'].head(10)

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7    0.129
8    0.129
9    0.129
Name: txpq, dtype: float64

In [5]:
# Function to fill in missing values by taking median of cross-section.
def fillMedianDataframe(df, date):
    # type(df) = pandas DataFrame
    # type(date) = str
    df_fill_median = df.copy()
    df_fill_median = df_fill_median.fillna(df.groupby(date).transform('median'))
    return df_fill_median

# Using function to fill in the missing values in the dataframe. We do this by taking median of cross-section.
date = 'datacqtr'
df_fill_median = fillMedianDataframe(df_ffill, date)

# Verifying output of the function.
df_fill_median['txpq'].head(10)

/var/folders/z9/c7h07wr544jdchl5xb1cvffm0000gq/T/ipykernel_96883/695323253.py:6: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the transforming function.
  df_fill_median = df_fill_median.fillna(df.groupby(date).transform('median'))


0       NaN
1       NaN
2       NaN
3    0.2060
4    0.2690
5    0.5845
6    0.6580
7    0.1290
8    0.1290
9    0.1290
Name: txpq, dtype: float64

In [6]:
# Function to add lags in the dataframe.
def createLaggedDataframe(df, id, lags_list):
    # type(df) = pandas DataFrame
    # type(id) = str
    # type (lags_list) = list
    df_lagged = df.copy()
    for lag in lags_list:
        columns_list = df.columns.tolist()
        for column in columns_list:
            new_col_name = column + str('_l') + str(lag) #FIXME: adapt so that shifts with pos int have neg int in name and vice versa
            df_lagged[new_col_name] = (df_lagged.groupby(id)[column].shift(lag))
    return df_lagged

# Using function to add lags (5 backward and 1 forward) in the dataframe.
id = 'cusip'
lags_list = [5,4,3,2,1,-1]
df_lagged = createLaggedDataframe(df_fill_median, id, lags_list)

# Verifying output of the function.
filter_col = [col for col in df_lagged if col.startswith('revty')]
df_lagged[filter_col].head()

/var/folders/z9/c7h07wr544jdchl5xb1cvffm0000gq/T/ipykernel_96883/817547854.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_lagged[new_col_name] = (df_lagged.groupby(id)[column].shift(lag))


,revty,revty_l5,revty_l4,revty_l3,revty_l2,revty_l1,revty_l-1
0,9.478,NaN,NaN,NaN,NaN,NaN,9.478
1,9.478,NaN,NaN,NaN,NaN,9.478,9.478
2,9.478,NaN,NaN,NaN,9.478,9.478,9.478
3,9.478,NaN,NaN,9.478,9.478,9.478,7.983
4,7.983,NaN,9.478,9.478,9.478,9.478,7.983


In [7]:
# Verifying that the original df remains unchanged.
filter_col = [col for col in df if col.startswith('revty')]
df[filter_col].head()

,revty
0,9.478
1,NaN
2,NaN
3,NaN
4,7.983


In [ ]:
#FIXME: define all functions first (together in a block) and show how they work with small and easy example. Then, chain everything together for the actual dataset.